# Summarize wifi results

In [5]:
import torch
import numpy as np
import os
import sys
from os.path import join
sys.path.append('../code')
from models import DAG_Generator
from sklearn.model_selection import cross_val_score
from sklearn import svm
import utils

In [6]:
def svm_accuracy(file_state, file_gen):
    
    config = torch.load(file_state)
    config = config['config']
    # print(config)

    input_dim = config['idim']
    num_class = config['num_class']
    num_domain = config['num_domain']
    dim_class = config['dim_y']
    dim_domain = config['dim_d']
    dim_hidden = config['dim_z']
    G_num_layer = config['G_mlp_layers']
    G_num_nodes = config['G_mlp_nodes']
    D_num_layer = config['D_mlp_layers']
    D_num_nodes = config['D_mlp_nodes']
    dag_mat_file = join(config['data_root'], config['dataset'], config['dag_mat_file'])
#     print(dag_mat_file)
    npzfile = np.load(dag_mat_file)
    dag_mat = npzfile['mat']
    isProb = config['estimate'] == 'Bayesian'
    gen = DAG_Generator(input_dim, num_class, num_domain, dim_class, dim_domain, dim_hidden, G_num_layer,
                                         G_num_nodes, False, dag_mat, prob=isProb)

    batch_size = 100
    device = 'cpu'
    gen.load_state_dict(torch.load(file_gen, map_location=torch.device(device)))

    label_d = torch.ones(batch_size*num_class) * (num_domain-1)
    label_y = torch.ones(batch_size) * 0
    for j in range(1, num_class):
        label_y = torch.cat((label_y, torch.ones(batch_size) * j), 0)


    if dim_hidden != 0:
        noise = torch.randn((label_d.shape[0], dim_hidden * input_dim), device=device)

    # create domain labels
    y_a_onehot = torch.nn.functional.one_hot(label_y.long(), num_class).float()

    d_onehot = torch.nn.functional.one_hot(label_d.long(), num_domain).float()

    if config['estimate'] == 'ML':
        fake_x = gen(noise, y_a_onehot, d_onehot, device=device)
    elif config['estimate'] == 'Bayesian':
        noise_d = torch.randn(num_domain, dim_domain * input_dim).to(device)
        fake_x = gen(noise, y_a_onehot, d_onehot, device=device, noise_d=noise_d)

    # train svm
    gamma_range = [2**-1, 1]
    C_range = [1, 10, 100]

    scores = np.zeros((len(gamma_range), len(C_range)))
    for i in range(len(gamma_range)):
        for j in range(len(C_range)):
            clf = svm.SVC(gamma=gamma_range[i], C=C_range[j])
            scores[i, j] = cross_val_score(clf, fake_x.detach().numpy(), label_y, cv=5).mean()

    (gamma_opt_id, C_opt_id) = np.unravel_index(np.argmax(scores), (len(gamma_range), len(C_range)))
    clf = svm.SVC(gamma=gamma_range[gamma_opt_id], C=C_range[C_opt_id], probability=False)
    clf.fit(fake_x.detach().numpy(), label_y.long())

    # predict
    test_dataset_specs = {'class_name': config['dataset'], 'seed': config['seed'], 'train': False,
                              'root': join(config['data_root'], config['dataset']), 'num_train': config['num_train'],
                              'num_domain': config['num_domain'], 'num_class': config['num_class'], 'dim': config['idim'],
                              'dim_d': config['dim_d'], 'dag_mat_file': config['dag_mat_file'], 'useMB': config['useMB'],
                              'tar_id': config['tar_id']}
    test_loader = utils.get_data_loader(test_dataset_specs, batch_size, config['num_workers'])
    x_t = test_loader.dataset.data
    y_t = test_loader.dataset.labels[:,0] * 2.0 / 3
    y_t = y_t.astype(int)
    y_pred = clf.predict(x_t)
    acc = sum(y_pred == y_t) * 1.0 / len(y_t)
    return acc

## tot1, NN classifier

In [7]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId1_seed%d_idim17_DA_Poolnn_ML_m0_warmup200_MLP_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden17_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId1_seed%d_idim17_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMM_DAG_Bayesian')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())

poolNN
best: 0.42319672212453535 0.01494919687697718
final: 0.38116248660372004 0.014475586693897213
JMM_DAG_Bayesian
best: 0.7295189968249364 0.019355281798218014
final: 0.7082111266920432 0.026974798323588053


## tot1, SVM classifier

In [8]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_state = '../weights/DatasetWifi_bk1/_tarId1_seed%d_idim17_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    file_gen = '../weights/DatasetWifi_bk1/_tarId1_seed%d_idim17_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_gen.pkl' % (i+1)
    acc_final[i] = svm_accuracy(file_state, file_gen)
print('JMM_DAG_Bayesian')
print('svm:', acc_final.mean(), acc_final.std())

dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
JMM_DAG_Bayesian
svm: 0.719475120557045 0.029362326412849065


## tot2, NN classifier

In [9]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId2_seed%d_idim16_DA_Poolnn_ML_m0_warmup200_MLP_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())

# for i in range(num_rep):
#     file_name = '../weights/DatasetWifi/_tarId2_seed%d_idim17_DA_Infer_JMMD_DAG_ML_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes100_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
#     x = torch.load(file_name)
#     acc_best[i] = x['best_score']
#     acc_final[i] = x['final_score']
# print('JMM_DAG_ML')
# print('best:', acc_best.mean(), acc_best.std())
# print('final:', acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId2_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMM_DAG_Bayesian')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())

poolNN
best: 0.4970426585196625 0.008401184173918991
final: 0.35117593842013084 0.022630607503720056
JMM_DAG_Bayesian
best: 0.8614325421818076 0.032772012329864285
final: 0.8454058120789526 0.02851198670519482


## tot2, SVM classifier

In [10]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_state = '../weights/DatasetWifi_bk1/_tarId2_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    file_gen = '../weights/DatasetWifi_bk1/_tarId2_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_gen.pkl' % (i+1)
    acc_final[i] = svm_accuracy(file_state, file_gen)
print('JMM_DAG_Bayesian')
print('svm:', acc_final.mean(), acc_final.std())

dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
JMM_DAG_Bayesian
svm: 0.8359636973099981 0.024976460052476647


## tot3, NN classifier

In [11]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId3_seed%d_idim16_DA_Poolnn_ML_m0_warmup200_MLP_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())


for i in range(num_rep):
    file_name = '../weights/DatasetWifi_bk1/_tarId3_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMM_DAG_Bayesian')
print('best:', acc_best.mean(), acc_best.std())
print('final:', acc_final.mean(), acc_final.std())

poolNN
best: 0.416217690500864 0.021060729405462664
final: 0.39562359616035814 0.02444649927241329
JMM_DAG_Bayesian
best: 0.8473169652978321 0.06787557367710828
final: 0.8298888990973184 0.07294657755692738


## tot3, SVM classifier

In [12]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_state = '../weights/DatasetWifi_bk1/_tarId3_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    file_gen = '../weights/DatasetWifi_bk1/_tarId3_seed%d_idim16_DA_Infer_JMMD_DAG_Bayesian_m0_warmup200_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight0.00_TAR_weight1.00_useMB1_G_mlp_nodes32_D_mlp_nodes64_bs210_Glr1.0e-02_Dlr1.0e-02_numDomain3_dimDomain1_dimHidden1_gen.pkl' % (i+1)
    acc_final[i] = svm_accuracy(file_state, file_gen)
print('JMM_DAG_Bayesian')
print('svm:', acc_final.mean(), acc_final.std())

dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
dataset=DatasetWifi(conf)
JMM_DAG_Bayesian
svm: 0.8374313026664192 0.06651531861687349
